In [3]:
import datetime
import cPickle as pickle
import csv
import numpy as np
import codecs
import re, sys
from konlpy.tag import Twitter
twitter = Twitter()

In [8]:
def is_dialogue_style(body):
    ds_list = ['입니다.', '습니다.',  '합니다.', '됩니다.']
    if body.find('입니다.') != -1:
        if body.count('입니다.') == 1 and body.find('기사입니다.') != -1:
            return 0
        else:
            return 1
    if body.find('습니다.') != -1:
        if body.count('습니다.') == 1 and body.find('권리가 있습니다.') != -1:
            return 0
        else:
            return 1
    if body.find('합니다.') != -1:
        if body.count('합니다.') == 1 and body.find('재배포 가능합니다.') != -1:
            return 0
        else:
            return 1
    if body.find('됩니다.') != -1:
        if body.count('됩니다.') == 1 and body.find('게제됩니다.') != -1:
            return 0
        else:
            return 1
    return 0

#### society case

In [11]:
filename = '../anaysis/201801-04_daum_society.csv'
# filename = '../anaysis/201801-04_daum_entertain.csv'
csv_reader = csv.reader(open(filename, 'r'))
articles = []
for n, row in enumerate(csv_reader):
    if n == 0: continue
    articles.append([n-1, row[5], row[6]])
        
cln_articles = []
n = 0
for item in articles:
    paras = [p for p in item[2].split('<p>') if p.strip()]
    if len(paras) < 5: continue
    body = '\n\n'.join(paras[2:-2])
    if is_dialogue_style(body): continue
    cln_articles.append([n, item[1], body])
    n += 1

#### entertain case

In [ ]:
# filename = '201801-04_daum_society.csv'
filename = '../anaysis/201801-04_daum_entertain.csv'
csv_reader = csv.reader(open(filename, 'r'))
articles = []
for n, row in enumerate(csv_reader):
    if n == 0: continue
    articles.append([n-1, row[5], row[2]])
    
cln_articles = []
print_articles = []
n = 0
for item in articles:
    paras = [p for p in item[2].split('<p>') if p.strip()]
    if len(paras) < 5: continue
    body = '\n\n'.join(paras[2:-2])
    ori_body = '\n\n'.join(paras)
    if is_dialogue_style(body): continue
    print_articles.append([n, item[1], ori_body])
    cln_articles.append([n, item[1], body])
    n += 1

# save csv file

In [8]:
csv_writer = csv.writer(open('tmp_' + filename, 'w'))
for row in cln_articles:
    csv_writer.writerow(row)

# functions

In [9]:
def llprint(message):
    sys.stdout.write(message)
    sys.stdout.flush()

In [10]:
def tokenize(article_list):
    cnt = 0   
    llprint('  >> Tokenizing dataset...\n')
    
    tkn_articles = []
    for article in article_list:
        cnt += 1
        idx = article[0]
        # head
        head = article[1].decode('utf-8')
        tag_head = twitter.morphs(head)
        tkn_head = []
        for tkn in tag_head:
            tkn_head.append(tkn)
        tkn_head = ' '.join(tkn_head)

        # body
        body = article[2]
        tkn_body = []
        for tap in body.split('\n\n'):
            if not tap:
                continue
            else:
                eos_list = re.finditer('[가-힣]+\.\s?|[가-힣]+\s?\)\.\s?', tap)
                prev_eos = 0
                for eos in eos_list:
                    tkn_sent = []
                    sentence = tap[prev_eos:eos.end()]
                    prev_eos = eos.end()
                    tag_sent = twitter.morphs(sentence.decode('utf-8'))
                    for tkn in tag_sent:
                        tkn_sent.append(tkn)
                    tkn_sent.append(u'<EOS>')
                    tkn_body.extend(tkn_sent)
                if prev_eos < len(tap):
                    tkn_sent = []
                    sentence = tap[prev_eos:]
                    tag_sent = twitter.morphs(sentence.decode('utf-8'))
                    for tkn in tag_sent:
                        tkn_sent.append(tkn)
                    tkn_sent.append(u'<EOS>')
                    tkn_body.extend(tkn_sent)
            tkn_body.append(u'<EOP>')
        tkn_body = ' '.join(tkn_body)
        nn_tkn = [idx, tkn_head.encode('utf-8'), tkn_body.encode('utf-8')]
        tkn_articles.append(nn_tkn)
    
    return tkn_articles

In [11]:
def fit_length(data, max_len_t, max_len_b):
    llprint('  >> Fitting length...\n')
    data_t, data_b = data
    list_zeros = np.zeros(max_len_b, 'int32').tolist()
    fl_data_t = []
    for datum in data_t:
        try:
            datum = list(datum)
        except:
            pass
        _len = len(datum)
        if _len >= max_len_t:
            fl_data_t.append( datum[:max_len_t] )
        else:
            fl_data_t.append( datum + list_zeros[:(max_len_t-_len)] )           
    fl_data_b = []
    for datum in data_b:
        try:
            datum = list(datum)
        except:
            pass
        _len = len(datum)
        if _len >= max_len_b:
            fl_data_b.append( datum[:max_len_b] )
        else:
            fl_data_b.append( datum + list_zeros[:(max_len_b-_len)] )
    np_data_t = np.asarray(fl_data_t, dtype='int32')
    np_data_b = np.asarray(fl_data_b, dtype='int32')   
    data = [np_data_t, np_data_b]
    return data

In [ ]:
tkn_articles = tokenize(cln_articles)

  >> Tokenizing dataset...


# para

In [ ]:
def encoding_testset(tkn_articles, vocab_path):
    llprint('  >> Converting words to idx...\n')
    tkn2idx = pickle.load(open(vocab_path, 'rb'))
    str_ids = []
    en_heads = []
    en_paras = []
    for _id, _head, _body in tkn_articles:
        head_encodings= []
        for tkn in _head.split():
            if tkn in tkn2idx:
                head_encodings.append(tkn2idx[tkn])
            else:
                head_encodings.append(1) # DO NOT CHANGE: index of <UNK> 
        for para in _body.split('<EOP>'):
            if para != ' ' and para:
                para = para + '<EOP>'
                para_encodings = []
                for tkn in para.split():
                    if tkn != ' ' and tkn:
                        if tkn in tkn2idx:
                            para_encodings.append(tkn2idx[tkn])
                        else:
                            para_encodings.append(1) # DO NOT CHANGE: index of <UNK> 
                str_ids.append(str(_id))
                en_heads.append(head_encodings)
                en_paras.append(para_encodings)
    if len(str_ids) != len(en_heads) and len(str_ids) != len(en_paras):
        raise NotImplementedError('Please Check Some Codes')
        
    return str_ids, en_heads, en_paras

In [ ]:
vocab_path =  '../dic_mincutN.pkl'
output_path = 'test_para_' + filename + '.pkl'
max_len_head = 49
max_len_para = 170

In [ ]:
str_ids, en_heads, en_paras = encoding_testset(tkn_articles, vocab_path)
[np_heads, np_paras] = fit_length([en_heads, en_paras], max_len_head, max_len_para)
llprint('  >> Saving preprocessed dataset in (%s)...\n' % output_path)
pickle.dump([str_ids, np_heads, np_paras], open(output_path,'wb'))

  >> Converting words to idx...
  >> Fitting length...
  >> Saving preprocessed dataset in (test_para_201801-04_daum_society.csv.pkl)...


# whole

In [ ]:
def encoding_whole_testset(tkn_articles, vocab_path):
    llprint('  >> Converting words to idx...\n')
    tkn2idx = pickle.load(open(vocab_path, 'rb'))
    str_ids = []
    en_heads = []
    en_bodys = []
    for _id, _head, _body in tkn_articles:
        head_encodings = []
        for tkn in _head.split():
            if tkn in tkn2idx:
                head_encodings.append(tkn2idx[tkn])
            else:
                head_encodings.append(1) # DO NOT CHANGE: index of <UNK> 
        body_encodings = []
        for tkn in _body.split():
            if tkn != ' ' and tkn:
                if tkn in tkn2idx:
                    body_encodings.append(tkn2idx[tkn])
                else:
                    body_encodings.append(1) # DO NOT CHANGE: index of <UNK> 
        str_ids.append(_id)
        en_heads.append(head_encodings)
        en_bodys.append(body_encodings)
    if len(str_ids) != len(en_heads) and len(str_ids) != len(en_bodys):
        raise NotImplementedError('Please Check Some Codes')
        
    return str_ids, en_heads, en_bodys

In [ ]:
vocab_path =  '../dic_mincutN.pkl'
output_path = 'test_whole_' + filename + '.pkl'
max_len_head = 49
max_len_body = 1000

In [ ]:
str_ids, en_heads, en_bodys = encoding_whole_testset(tkn_articles, vocab_path)
[np_heads, np_bodys] = fit_length([en_heads, en_bodys], max_len_head, max_len_body)
llprint('  >> Saving preprocessed dataset in (%s)...\n' % output_path)

  >> Converting words to idx...
  >> Fitting length...
  >> Saving preprocessed dataset in (test_whole_201801-04_daum_society.csv.pkl)...


In [ ]:
pickle.dump([str_ids, np_heads, np_bodys], open(output_path,'wb'))

# history

In [20]:
import random

In [21]:
print len(cln_articles),
i = random.randint(0, len(cln_articles))
print i
print cln_articles[i][1]
print cln_articles[i][2]

578269 136129
유행렬 청와대 행정관 "청주시장 출마 긍정적으로 고민 중"
그는 "이제는 청와대에 남아 지역발전을 위해 노력할 것인 지 아니면 청주시장 출마가 지역에 도움이 되는 지 등으로 가닥을 잡고 생각하고 있다"고 말했다.

이어 "이번 설 명절 때 가족들과 상의해 결정하려 한다"며 "가족들이 적극적으로 협조를 해야 저도 편하게 결정하고 (출마를)실행할 수 있다"고 강조했다.

최근 한 언론사의 여론조사에서 1위에 오른 것이 '외부 효과'란 지적과 관련해 "지금의 현재 모습으로 그 정도 평가가 나온 것"이라며 선을 분명히 그었다.

유 행정관은 "저는 20여 년 동안 지역에서 오랜 인맥을 만들고 바닥에서부터 실무를 다져왔다"며 "청와대에서 근무하게 된 것도 실력이 있다는 검증을 받은 것"이라고 강조했다.

또 "사람을 평가할 때 현재가 과거와 분리돼 평가받는 것이 아니라 총체적인 것으로 생각한다"며 "인지도와 지지도는 개념 자체가 다르며 지금 모습으로 충분히 평가가 나온 것"이라고 주장했다.

'문 대통령의 최측근으로 분류되는 노영민 주중대사와 교감이 있었냐'는 질문에는 "(서로)어려운 문제가 있으면 상의하지만 허락을 하고 받는 관계는 아니다"라고 말했다.


In [22]:
csv_reader = csv.reader(open('tmp_' + filename, 'r'))
articles = []
for n, row in enumerate(csv_reader):
    if n == 0: continue
    articles.append([row[0], row[1], row[2]])

In [23]:
# open test.pkl file
[str_ids, np_heads, np_paras] = pickle.load(open('../test.pkl', 'rb'))

In [24]:
len(str_ids)

5699

In [25]:
csv_reader = csv.reader(open(filename, 'r'))
articles = []
for n, row in enumerate(csv_reader):
    if n == 0: continue
    articles.append([n-1, row[5], row[6]])

In [26]:
print len(articles)
print articles[0][1]
print articles[0][2]

1309163
[제천소식]이시종 충북지사 제천서 장보기 등
<p>【제천=뉴시스】강신욱 기자 = ◇이시종 충북지사 제천서 장보기<p> <p>이시종 충북도지사는 14일 화재 참사로 어려움을 겪는 제천을 방문했다.<p> <p>이 지사는 이날 내토시장을 찾아 설 명절을 앞두고 장보기 행사를 했다.<p> <p>이 지사는 제천소방서를 방문해 직원들을 격려하고 사회복지시설을 찾아 어르신들을 위로했다.<p> <p>이어 충주로 이동해 1961년 준공된 구 목행대교에 대한 국가안전대진단 특별점검을 했다.<p> <p>◇제천소방서, 안전 홍보 캠페인<p> <p>충북 제천소방서(서장 김상현)는 14일 오후 제천역전시장과 시외버스터미널에서 '고향집, 주택용 소방시설 선물하기 홍보 캠페인'을 했다.<p> <p>이날 캠페인은 주택용 소방시설 설치 촉진과 자율 설치 환경 조성을 위해 귀성객을 대상으로 진행했다.<p> <p>제천소방서는 소방시설(소화기·감지기) 선물과 소방차 길 터주기 캠페인도 함께했다.◇세명대 간호학과 국가고시 100％ 합격<p> <p>충북 제천 세명대학교는 간호학과가 간호사 국가고시에 100％에 합격했다고 14일 밝혔다.<p> <p>세명대 간호학과는 최근 한국보건의료인국가시험원에서 시행한 58회 간호사 국가고시에 졸업예정자 108명이 응시했다.<p> <p>이번 졸업자 96.3％가 강남세브란스병원, 서울대병원, 삼성서울병원, 서울아산병원, 서울성모병원 등에 취업했다.<p> <p>ksw64@newsis.com<p>


In [27]:
i = random.randint(0, len(articles))
print i
print articles[i][1]
print articles[i][2]

341058
김소희, '이윤택 성폭력' 피해자의 "등 떠밀었다" 폭로에 "사실 아니다"
<p>지난 19일 '뉴스룸'에서는 이윤택씨에게 성폭력을 당했다고 주장한 피해자가 실명을 밝히지 않고 목소리를 변조해 손석희 앵커와 인터뷰를 가졌다. 피해자는 "2004, 2005년 정도부터 성폭력 피해를 입었다"며 "안마라는 이름으로 수위를 넘어서는 행위를 강요받았다"고 주장했다.<p> <p>이어 피해자는 “(이윤택이) 나는 너와 자고 싶다”며 가슴으로 손이 쑥 들어와 급하게 피한 적도 있고 “발성을 키워야 된다면서 사타구니 쪽에 나무젓가락을 꽂은 적도 있었다”고 폭로했다.<p> <p>피해자는 "안마를 거부하면 전체 단원을 모은 뒤 거부한 한명을 두고 마녀사냥하듯 거부한 여자 단원에 대한 안 좋은 점을 이야기했다. 그 전에 캐스팅됐던 역할에서 배제시켰다"고 밝혔다.<p> <p>이어 이 피해자는 "나에게 ‘이윤택이 안마를 원한다 들어가라’며 등을 떠민 건 여자선배였다"고 증언했다. 그는 "김소희 대표가 조력자처럼 후배를 초이스하고 안마를 권유했다"며 "나에게 과일이 든 쟁반을 주면서 이윤택 방에 가서 안마를 하라고 했다. 내가 거부하자 가슴팍을 치면서 왜 이렇게 이기적이냐 너만 희생하면 되는데 왜 그러냐고 말했다. 아직까지 그 눈빛이 잊혀지지 않는다"고 주장했다.<p> <p>이에 대해 김 대표는 19일 지인의 SNS를 통해 "저희 극단이 잘못한 일로 책임감은 크지만 JTBC 뉴스에 나온 내용은 전혀 사실이 아니다"고 밝혔다. 연희단거리패는 이날 해체 발표 이후 공식 홈페이지와 SNS를 모두 폐쇄해 지인의 SNS를 통해 입장을 밝힌 것으로 보인다.<p> <p>김소희 대표는 이 같은 증언에 대해 "저도 너무 놀라 손이 떨린다. 방송국 측에 정정신청 해놓았다"고 밝혔다. 이어 "인터뷰한 사람이 누군지 모르겠지만 사실을 밝히는 데 필요한 조치가 있다면 다 할 것이다"고 덧붙였다.<p>


In [28]:
cleaned_articles = []
n = 0
for item in articles:
    paras = [p for p in item[2].split('<p>') if p.strip()]
    if len(paras) < 3: continue
    body = '\n\n'.join(paras[1:-1])
    cleaned_articles.append([n, item[1], body])

In [29]:
i = random.randint(0, len(cleaned_articles))
print i
print cleaned_articles[i][1]
print cleaned_articles[i][2]

608249
교통사고 사망자 5년 연속 감소..보행자 사망 사고 비율은 여전히 높아
6일 경찰청이 발표한 2017년 교통사고 분석에 따르면 작년 한 해 교통사고 사망자는 4185명으로 전년(4292명) 대비 2.5%(107명) 줄어들었다. 교통사고 사망자는 지난 2012년 5392명에서 지난해까지 5년 연속 감소했다.

교통사고 사망자는 감소세를 나타내고 있지만 보행자 사망 비율은 여전히 높은 것으로 나타났다. 지난해 교통사고로 숨진 보행자는 1675명으로 전체 사망사고에서 40%를 차지했다. 2016년 기준 경제협력개발기구(OECD) 보행 사망자 비율(19.2%)의 두 배를 웃도는 수치다. 특히 노인 보행자 사망은 906명으로 전년보다 40명(4.6%) 증가해 전체 보행자 사망의 54.1%에 달했다.

경찰 관계자는 "OECD 등 선진국에서 교통사고 사망자수 감소에 성과를 거둔 '안전속도 5030' 시범운영을 확대하는 등 보행자 친화적인 정책을 시행할 것"이라며 "2012년 이후 교통사고 사망자가 지속해서 감소하고 있고 올해는 3000명대로 떨어질 것으로 기대된다"고 전했다. 안전속도5030은 도시 내 제한속도를 50km/h로 하고, 보호구역 등 특별보호 필요 지역은 30km/h로 지정하는 속도관리정책이다.

최근 3년간 증가했던 어린이 사망자는 지난해 54명으로 전년(71명)보다 17명(23.9%) 줄었다. 스쿨존(어린이보호구역) 사망자는 전년과 같은 8명이었으나 부상자는 23명 감소한 487명이었고, 통학버스 사망자는 없었다.

음주운전 사망자는 439명으로 전년(481명)보다 8.7%(42명) 감소하는 등 2012년 이후 지속적으로 감소세를 보인 것으로 나타났다.


In [ ]:
csv_reader = csv.reader(open('20180501-02_naver.csv', 'r'))
articles = []
for n, row in enumerate(csv_reader):
    if n == 0: continue
    articles.append([n-1, row[4], row[5]])

In [ ]:
print len(articles)
print articles[0][1]
print articles[0][2]

In [ ]:
i = random.randint(0, len(articles))
print i
print articles[i][1]
print articles[i][2]

In [ ]:
csv_reader = csv.reader(open('20180501-02_daum.csv', 'r'))



In [ ]:
pickle.dump([str_ids, np_heads, np_paras], open(output_path,'wb'))
